In [1]:
import pandas as pd

In [2]:
access = pd.read_csv("/home/jaime/projet_info_3A/3A-ENSAE-projet-info/private/access.csv",header=0)
id_api = access["Identifiant"][0]
key_api = access["Clé secrète"][0]

In [3]:
import requests 
import os

In [4]:
import emploi_store

In [5]:
client_id = id_api
client_secret = key_api
access_tokens = {}
cached_packages = None
scope = 'api_offresdemploiv2'

In [6]:
# from https://github.com/Cocorico84/employme/blob/855cfcda2c54f42b9c42dcbf4ac87cdf09248b7f/back/manager.py

In [7]:
import requests


url = "https://api.emploi-store.fr/partenaire"

In [8]:
def get_token():
    url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token'
    r = requests.post(url_token, headers={'Content-Type': 'application/x-www-form-urlencoded'},
                      params={'realm': '/partenaire'},
                      data={'grant_type': 'client_credentials', 'client_id': id_api,
                            'client_secret': key_api,
                            'scope': f'application_{id_api} api_offresdemploiv2 o2dsoffre'})
    data = r.json()
    return data['access_token']


In [9]:
def connection(token):
    url_offers = f'{url}/offresdemploi/v2/offres/search'
    r = requests.get(url=url_offers, #,  params={'motsCles': 'informatique'}
        headers={'Authorization': f'Bearer {get_token()}'})
    data = r.json()

    return data

In [10]:
request_as_dict=connection(get_token())

In [14]:
df = pd.DataFrame.from_dict(request_as_dict['resultats'])


In [20]:
df.to_csv("first_test.csv", sep=",", header=False, index=False)

In [18]:
df.iloc[0]["description"]

"Consultant chez Fed Finance, je recherche pour l'un de mes clients, cabinet d'expertise comptable, un Gestionnaire Paie (H/F) en CDI a\xa0 Fecamp (76403). Qu'est-ce que le metier de Collaborateur social Votre fonction VOTRE NCTION Envie de rejoindre une equipe dynamique au sein d'un cabinet d'une quinzaine de personnes. Voici ce que nous proposons : sous la responsabilite du responsable du pole social, vous intervenez sur un portefeuille multi-conventionnel. Vos missions sont les suivantes : - La gestion de l'entree a\xa0 la sortie du salarie : de travail, DPAE, DSN, attestations diverses, rupture conventionnelle, demission, licenciement ; - La saisie des bulletins de paie de vos client (300 a\xa0 350 par mois) ; - L'accompagnement des clients dans leurs differentes problematiques liees a\xa0 leurs salaries ; Ce est aussi l'occasion pour vous de monter en competences. Votre profil VOTRE PROFIL De formation en Ressource Humaine ou oriente vers la gestion de la paie (type BAC2/3), vous 